In [ ]:
import tensorflow as tf

from pod.ai.manual import DQController
from pod.board import PodBoard
from pod.controller import Controller
from pod.drawer import Drawer
from pod.game import Player
from pod.util import PodState
from vec2 import Vec2
from IPython.display import Image


board = PodBoard([Vec2(5000, 4000), Vec2(12000, 7000)])
controller = DQController(board)
pods = controller.gen_pods(5,5,5,5,5)

In [ ]:
for epoch in range(1, 4):
    print("----- stupid epoch {} -----".format(epoch))
    controller.train(pods, batch_size=30, use_best = True)

In [ ]:
controller.train(pods, batch_size=70, discount=0.2, epsilon=0.9)

In [ ]:
player = Player(controller, pod=PodState((board.get_check(0) + board.get_check(1)) * 0.5))
player.pod.angle = 1.23
drawer = Drawer(board, [player])

file = '/tf/notebooks/pods.gif'
drawer.animate(file, 50)
Image(filename = file)

In [ ]:
import numpy as np
import tensorflow as tf
from pod.ai.ai_utils import THRUST_VALUES, ANGLE_VALUES

BATCH_SIZE = 100

model = tf.keras.Sequential([
    tf.keras.layers.Dense(
        100,
        input_shape=(6,),
        activation="tanh"
    ),
    tf.keras.layers.Dense(
        100,
        activation="tanh"
    ),
    tf.keras.layers.Dense(
        THRUST_VALUES * ANGLE_VALUES
    )
])

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    loss=tf.keras.losses.mean_squared_error
)

target = np.zeros((BATCH_SIZE, THRUST_VALUES * ANGLE_VALUES))
target[...,0] = 1
for epoch in range(1, 100):
    batch = np.random.rand(BATCH_SIZE, 6)
    loss = model.train_on_batch(batch, target)
    print("Epoch {} loss {}".format(epoch, loss))

test_x = np.random.rand(1, 6)
test_y = model(tf.constant(test_x))
print("test_x = {}".format(test_x))
print("test_y = {}".format(test_y))